In [ ]:
import numpy as np
import pandas as pd
import requests
import random

# FVtyMiv9TXSjy0x4O9xmVYE2iksnAlYeEXPaD2sm

In [ ]:
eia_api_key = 'FVtyMiv9TXSjy0x4O9xmVYE2iksnAlYeEXPaD2sm'

In [ ]:
# api_url = f'https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={eia_api_key}'

# headers = {
#     "X-Params": '{"frequency": "hourly", "data": ["value"], "facets": {}, "start": "2022-10-01T00", "end": "2023-10-01T00", "sort": [{"column": "period", "direction": "desc"}], "offset": 0, "length": 5000}'
# }

# response = requests.get(api_url, headers=headers)

# if response.status_code == 200:
#     data = response.json()

# else:
#     print(f"Error: {response.status_code} - {response.text}")

In [ ]:
# locations = []
# types = [item['type-name'] for item in data["response"]["data"] if item['type-name'] == 'Demand']

# Demand = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Demand']

# Total_interchange = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Total Interchange']

# Net_generation = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Net Generation']

# Day_ahead = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Day-ahead Demand Forecast']



# all_values = [item["type-name"] for item in data["response"]["data"]]


# desired_responses = ["Day-ahead demand forecast", "Net generation", "Demand", "Total interchange"]

# # Create separate lists for each response
# response_data = {}
# for response in desired_responses:
#     response_data[response] = [item["value"] for item in data["response"]["data"] if item['type-name'] == response]

# # Print the data for each response
# for response, values in response_data.items():
#     print(response)
#     for value in values:
#         print(value)
#     print('\n')

# # for value in all_values:
# #   print(value)

# # for i, (type, value) in enumerate(zip(types, Demand)):
# #   print(i, type, value)
# #   print('\n')
# #print(types, values)

In [ ]:
max_len = max(len(types), len(Demand), len(Total_interchange), len(Net_generation), len(Day_ahead))

# Pad lists with NaN to make them of the same length
def pad_list(lst, length):
    if len(lst) < length:
        return lst + [np.nan] * (length - len(lst))
    return lst

types = pad_list(types, max_len)
Demand = pad_list(Demand, max_len)
Total_interchange = pad_list(Total_interchange, max_len)
Net_generation = pad_list(Net_generation, max_len)
Day_ahead = pad_list(Day_ahead, max_len)

# Create the DataFrame
data = {
    'Type': types,
    'Demand': Demand,
    'Total Interchange': Total_interchange,
    'Net Generation': Net_generation,
    'Day-ahead Demand Forecast': Day_ahead
}

df = pd.DataFrame(data)

# Print the DataFrame
print(df)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("EIA930_BALANCE_2023_Jul_Dec(3).csv")
df = df.loc[:, 'Data Date':'Total Interchange (MW)']

In [ ]:
def generate_fake_cost():
    # Generate random values around $50 with some variation
    base_cost = 50
    variation = random.uniform(-20, 20) # Varaition, can adjust
    return base_cost + variation

# Create the 'Electricity Cost ($)' column with fake data
df['Electricity Cost ($)'] = [generate_fake_cost() for _ in range(len(df))]

In [ ]:
def calculate_charging_rate(electricity_cost, median_cost):
    #Calculate deviation from the median
    deviation = electricity_cost - median_cost
    
    #Charge faster when cost is below the median, slower when above
    if deviation < 0:
        return 1.0  # Charge at full rate
    else:
        return max(0.2, 0.2 + (1.0 - 0.2) * (1 - deviation / median_cost))  # Linearly decrease charging rate as cost deviates

# Calculate the median electricity cost
median_cost = df['Electricity Cost ($)'].median()

df['Charging Rate'] = df['Electricity Cost ($)'].apply(lambda x: calculate_charging_rate(x, median_cost))

In [ ]:
df.head()

In [ ]:
print(df["Charging Rate"].min())